In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import ipywidgets
from IPython.display import display
from collections import defaultdict

np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.float_format", lambda x: "{:.2f}".format(x))
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
# pd.set_option("display.min_rows", 10)

# (11.7, 8.27) = A4 landscape
sns.set_theme(style="dark", palette="muted", font_scale=1.10, rc={"figure.figsize": (16.54, 11.7)})

SELECTORS = ["w/o", "w/"]  # ordered as bools
MARGIN_SYMB = "Σ"
CLASS_SYMB = "*"
MARGIN_IDX = (CLASS_SYMB, MARGIN_SYMB, SELECTORS[1])
SORT_ORDER = [True, True, False]

# DATASET_FILENAME = Path("results/activities_2022-01-29_16-33-05.csv")
DATASET_FILENAME = Path("results/pharmaco_chemistry_2022-03-17_16-43-57.csv")
dataset_with_margin = pd.read_csv(DATASET_FILENAME, index_col=[0, 1, 2], header=[0, 1, 2])
dataset_with_margin.sort_index(axis=1, inplace=True, ascending=SORT_ORDER)
dataset_with_margin.sort_index(axis=0, inplace=True, ascending=SORT_ORDER)

N = dataset_with_margin.loc[MARGIN_IDX,MARGIN_IDX]

all_comp_but_margin = pd.Series(idx for idx in dataset_with_margin.index if idx != MARGIN_IDX)
all_acti_but_margin = pd.Series(idx for idx in dataset_with_margin.columns if idx != MARGIN_IDX)

comp_margin = dataset_with_margin.loc[all_comp_but_margin, MARGIN_IDX]
acti_margin = dataset_with_margin.loc[MARGIN_IDX, all_acti_but_margin]



dataset = dataset_with_margin.loc[all_comp_but_margin, all_acti_but_margin]

display(dataset)


abiotic 
 allelopathy 
 pharmaco 
 toxicity 
 
 
 
 
 
 antioxidant 
 drought 
 metal 
 salt 
 uv 
 antifeedant 
 arbuscula 
 attractant 
 germination 
 herbicidal 
 hppd 
 hyphal 
 phytotoxicity 
 quorum sensing 
 repulsive 
 toxicant 
 anti-inflammatory 
 antibacterial 
 anticancer 
 antidiabetic 
 antifungal 
 antimicrobial 
 antiparasitic 
 antiviral 
 arthritis 
 burns 
 cardiovascular 
 cytotoxicity 
 obesity 
 rheumatism 
 sedative 
 wound 
 toxicity 
 
 
 
 
 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 w/o 
 w/ 
 
 
 
 
 alkaloid 
 acridine 
 w/o 
 181187 
 63047 
 243137 
 1097 
 218950 
 25284 
 230892 
 13342 
 237674 
 6560 
 244035 
 199 
 244223 
 11 
 244108 
 126 
 241766 
 2468 
 244036 
 198 
 244219 
 15 
 244195 
 39 
 242992 
 1242 
 243327 
 907 
 244173 
 61 
 244118 
 116 
 227131 
 17103 
 224069 
 20165 
 239650 
 4584 
 238246 
 5988 
 229322 
 14912 
 222208 
 22026 
 243159 
 1075 
 235198 
 9036 
 236853 
 7381 
 243047 
 1187 
 234011 
 10223 
 204543 
 39691 
 239205 
 5029 
 244162 
 72 
 241740 
 2494 
 237393 
 6841 
 218279 
 25955 
 
 
 w/ 
 2452 
 271 
 2721 
 2 
 2464 
 259 
 2560 
 163 
 2643 
 80 
 2722 
 1 
 2723 
 0 
 2723 
 0 
 2712 
 11 
 2722 
 1 
 2723 
 0 
 2723 
 0 
 2715 
 8 
 2713 
 10 
 2723 
 0 
 2718 
 5 
 2660 
 63 
 2593 
 130 
 2574 
 149 
 2709 
 14 
 2644 
 79 
 2641 
 82 
 2697 
 26 
 2606 
 117 
 2697 
 26 
 2707 
 16 
 2675 
 48 
 1414 
 1309 
 2720 
 3 
 2723 
 0 
 2715 
 8 
 2644 
 79 
 2313 
 410 
 
 
 benzylamine 
 w/o 
 182861 
 63247 
 245009 
 1099 
 220732 
 25376 
 232685 
 13423 
 239491 
 6617 
 245908 
 200 
 246097 
 11 
 245982 
 126 
 243633 
 2475 
 245915 
 193 
 246093 
 15 
 246069 
 39 
 244860 
 1248 
 245191 
 917 
 246047 
 61 
 245988 
 120 
 228996 
 17112 
 225844 
 20264 
 241394 
 4714 
 240119 
 5989 
 231191 
 14917 
 224023 
 22085 
 245009 
 1099 
 236979 
 9129 
 238713 
 7395 
 244906 
 1202 
 235928 
 10180 
 205235 
 40873 
 241110 
 4998 
 246036 
 72 
 243612 
 2496 
 239207 
 6901 
 219845 
 26263 
 
 
 w/ 
 778 
 71 
 849 
 0 
 682 
 167 
 767 
 82 
 826 
 23 
 849 
 0 
 849 
 0 
 849 
 0 
 845 
 4 
 843 
 6 
 849 
 0 
 849 
 0 
 847 
 2 
 849 
 0 
 849 
 0 
 848 
 1 
 795 
 54 
 818 
 31 
 830 
 19 
 836 
 13 
 775 
 74 
 826 
 23 
 847 
 2 
 825 
 24 
 837 
 12 
 848 
 1 
 758 
 91 
 722 
 127 
 815 
 34 
 849 
 0 
 843 
 6 
 830 
 19 
 747 
 102 
 
 
 colchicine 
 w/o 
 178008 
 63121 
 240034 
 1095 
 215672 
 25457 
 227812 
 13317 
 234510 
 6619 
 240929 
 200 
 241118 
 11 
 241003 
 126 
 238697 
 2432 
 240930 
 199 
 241114 
 15 
 241090 
 39 
 239888 
 1241 
 240213 
 916 
 241068 
 61 
 241009 
 120 
 224861 
 16268 
 220880 
 20249 
 236570 
 4559 
 235228 
 5901 
 226271 
 14858 
 219069 
 22060 
 240044 
 1085 
 232087 
 9042 
 235367 
 5762 
 239958 
 1171 
 231506 
 9623 
 201393 
 39736 
 236272 
 4857 
 241077 
 52 
 238661 
 2468 
 234432 
 6697 
 215632 
 25497 
 
 
 w/ 
 5631 
 197 
 5824 
 4 
 5742 
 86 
 5640 
 188 
 5807 
 21 
 5828 
 0 
 5828 
 0 
 5828 
 0 
 5781 
 47 
 5828 
 0 
 5828 
 0 
 5828 
 0 
 5819 
 9 
 5827 
 1 
 5828 
 0 
 5827 
 1 
 4930 
 898 
 5782 
 46 
 5654 
 174 
 5727 
 101 
 5695 
 133 
 5780 
 48 
 5812 
 16 
 5717 
 111 
 4183 
 1645 
 5796 
 32 
 5180 
 648 
 4564 
 1264 
 5653 
 175 
 5808 
 20 
 5794 
 34 
 5605 
 223 
 4960 
 868 
 
 
 cyclopeptide 
 w/o 
 181009 
 63260 
 243171 
 1098 
 218894 
 25375 
 230800 
 13469 
 237645 
 6624 
 244069 
 200 
 244258 
 11 
 244143 
 126 
 241808 
 2461 
 244071 
 198 
 244254 
 15 
 244231 
 38 
 243032 
 1237 
 243416 
 853 
 244208 
 61 
 244148 
 121 
 227195 
 17074 
 224227 
 20042 
 239584 
 4685 
 238297 
 5972 
 229665

In [9]:
display(acti_margin)
display(comp_margin)
display(N)

abiotic      antioxidant        w/o    183639
                                w/      63318
             drought            w/o    245858
                                w/       1099
             metal              w/o    221414
                                w/      25543
             salt               w/o    233452
                                w/      13505
             uv                 w/o    240317
                                w/       6640
allelopathy  antifeedant        w/o    246757
                                w/        200
             arbuscula          w/o    246946
                                w/         11
             attractant         w/o    246831
                                w/        126
             germination        w/o    244478
                                w/       2479
             herbicidal         w/o    246758
                                w/        199
             hppd               w/o    246942
                                w/         15
             hyphal             w/o    246918
                                w/         39
             phytotoxicity      w/o    245707
                                w/       1250
             quorum sensing     w/o    246040
                                w/        917
             repulsive          w/o    246896
                                w/         61
             toxicant           w/o    246836
                                w/        121
pharmaco     anti-inflammatory  w/o    229791
                                w/      17166
             antibacterial      w/o    226662
                                w/      20295
             anticancer         w/o    242224
                                w/       4733
             antidiabetic       w/o    240955
                                w/       6002
             antifungal         w/o    231966
                                w/      14991
             antimicrobial      w/o    224849
                                w/      22108
             antiparasitic      w/o    245856
                                w/       1101
             antiviral          w/o    237804
                                w/       9153
             arthritis          w/o    239550
                                w/       7407
             burns              w/o    245754
                                w/       1203
             cardiovascular     w/o    236686
                                w/      10271
             cytotoxicity       w/o    205957
                                w/      41000
             obesity            w/o    241925
                                w/       5032
             rheumatism         w/o    246885
                                w/         72
             sedative           w/o    244455
                                w/       2502
             wound              w/o    240037
                                w/       6920
toxicity     toxicity           w/o    220592
                                w/      26365
Name: (*, Σ, w/), dtype: int64

alkaloid           acridine                             w/o    244234
                                                        w/       2723
                   benzylamine                          w/o    246108
                                                        w/        849
                   colchicine                           w/o    241129
                                                        w/       5828
                   cyclopeptide                         w/o    244269
                                                        w/       2688
                   imidazole                            w/o    230271
                                                        w/      16686
                   indole                               w/o    233401
                                                        w/      13556
                   indolizidine                         w/o    246884
                                                        w/         73
                   isoquinoline                         w/o    244905
                                                        w/       2052
                   isoxazole                            w/o    244454
                                                        w/       2503
                   muscarine                            w/o    246878
                                                        w/         79
                   oxazole                              w/o    245055
                                                        w/       1902
                   phenylethylamine                     w/o    246888
                                                        w/         69
                   piperidine                           w/o    240378
                                                        w/       6579
                   purine                               w/o    242039
                                                        w/       4918
                   putrescine                           w/o    245907
                                                        w/       1050
                   pyridine                             w/o    227588
                                                        w/      19369
                   pyrrolidine                          w/o    243691
                                                        w/       3266
                   pyrrolizidine                        w/o    246258
                                                        w/        699
                   quinazoline                          w/o    244202
                                                        w/       2755
                   quinoline                            w/o    233788
                                                        w/      13169
                   quinolizidine                        w/o    246807
                                                        w/        150
                   spermidine                           w/o    245680
                                                        w/       1277
                   spermine                             w/o    245638
                                                        w/       1319
                   thiazole                             w/o    234940
                                                        w/      12017
                   tropane                              w/o    246688
                                                        w/        269
phenolic compound  acetophenone                         w/o    244814
                                                        w/       2143
                   anthraquinone                        w/o    242970
                                                        w/       3987
                   biflavonoids                         w/o    246184
                                                        w/        773
                   flavonoids                           w/o    204481
          

246957

Boilerplate : juste de la manipulation de dictionnaire/MultiIndex

In [10]:
# # toutes les classes chimique, le 1er niveau de l'index
comp_class = sorted(set(dataset.index.get_level_values(0)))
# # dictionnaire qui à une molécule fait correspondre sa classe
# get_comp_class = {k: c for c, k in dataset.index.droplevel(2)}
# # le dictionnaire inverse du précédent : la liste des molécule d'une classe
get_comps = defaultdict(set)
for c, k in dataset.index.droplevel(2):
    get_comps[c].add(k)
#  le cas particulier la classe "tous"
# get_comps[ALL] = set(dataset.index.get_level_values(1))
get_comps[CLASS_SYMB] = set()
# on passe en listes une bonne fois pour toutes
get_comps = {k: list(l) for k, l in get_comps.items()}

# # idem pour les activités
acti_class = sorted(set(dataset.columns.get_level_values(0)))
# get_acti_class = {k: c for c, k in dataset.columns.droplevel(2)}
get_actis = defaultdict(set)
for c, k in dataset.columns.droplevel(2):
    get_actis[c].add(k)
# get_actis[ALL] = set(dataset.columns.get_level_values(1))
get_actis[CLASS_SYMB] = set()
get_actis = {k: list(l) for k, l in get_actis.items()}


# get_comps


In [11]:
def to_slice(x_class=CLASS_SYMB, x_name=CLASS_SYMB):
    """Turns a pair (class, name) into a slice. Must be consistent in the sens that if comp is set, comp_class should be its class."""
    if x_class == CLASS_SYMB:
        if x_name != CLASS_SYMB:
            raise ValueError(f"x_name ({x_name}) should be a refinement of x_class wich is {CLASS_SYMB}")
        return slice(None, None)
    elif x_name == CLASS_SYMB:
        return slice((x_class,), (x_class,))
    else:
        return slice((x_class, x_name), (x_class, x_name))


In [12]:
# avec loc et les slices on accède facilement à des ensemble de lignes/colonnes hiérarchisés
dataset.loc[to_slice("polyketide")]
# dataset.loc[to_slice(ALL, "ansamycin")]
comp_margin.xs(SELECTORS[1], level=2)


alkaloid           acridine                                2723
                   benzylamine                              849
                   colchicine                              5828
                   cyclopeptide                            2688
                   imidazole                              16686
                   indole                                 13556
                   indolizidine                              73
                   isoquinoline                            2052
                   isoxazole                               2503
                   muscarine                                 79
                   oxazole                                 1902
                   phenylethylamine                          69
                   piperidine                              6579
                   purine                                  4918
                   putrescine                              1050
                   pyridine                               19369
                   pyrrolidine                             3266
                   pyrrolizidine                            699
                   quinazoline                             2755
                   quinoline                              13169
                   quinolizidine                            150
                   spermidine                              1277
                   spermine                                1319
                   thiazole                               12017
                   tropane                                  269
phenolic compound  acetophenone                            2143
                   anthraquinone                           3987
                   biflavonoids                             773
                   flavonoids                             42476
                   isoflavanoids                              2
                   lignans                                 2459
                   naphthoquinone                          2841
                   phenol                                 54494
                   phenolic acid                           2442
                   phenylpropanoid                          856
                   stilbene                                5246
                   tannin                                 11321
                   xanthone                                1837
polyketide         acetogenin                               317
                   ansamycin                                119
                   macrolide                               6965
                   polyene                                 1438
                   polyether                                961
                   tetracycline                           17798
terpenoid/terpene  diterpene                               3448
                   hemiterpene                              162
                   monoterpene                              742
                   norisoprenoid                             77
                   polyterpene                                3
                   sesquiterpene                           5494
                   sesterterpene                            192
                   tetraterpene/carotenoid/xanthophyll    10707
                   triterpene                              5685
Name: (*, Σ, w/), dtype: int64

In [15]:
# OK
# ipywidgets.interact(
#     lambda comp_class: display(dataset.loc[comp_class]),
#     comp_class=ipywidgets.Dropdown(
#         options=comp_class,
#         value="polyketide",
#         description="Class",
#     ),
# )

comp_class_widget = ipywidgets.Dropdown(
    options=[CLASS_SYMB] + comp_class,
    value=CLASS_SYMB,
    description="C-Class",
)

comp_widget = ipywidgets.Dropdown(
    options=[CLASS_SYMB] + get_comps[comp_class_widget.value],
    value=CLASS_SYMB,
    description="Compound",
)

acti_class_widget = ipywidgets.Dropdown(
    options=[CLASS_SYMB] + acti_class,
    value=CLASS_SYMB,
    description="A-Class",
)

acti_widget = ipywidgets.Dropdown(
    options=[CLASS_SYMB] + get_actis[acti_class_widget.value],
    value=CLASS_SYMB,
    description="Activity",
)

# comp_margin_w = comp_margin.xs(SELECTORS[1], level=2)
# comp_threshold_widget = ipywidgets.IntSlider(min=comp_margin_w.min(), max=comp_margin_w.max(), step=10)
# acti_margin_w = acti_margin.xs(SELECTORS[1], level=2)
# 

THRESHOLD_MIN, THRESHOLD_MAX = 0, 10_000
comp_threshold_widget = ipywidgets.IntSlider(min=THRESHOLD_MIN, max=THRESHOLD_MAX, step=20)
acti_threshold_widget = ipywidgets.IntSlider(min=THRESHOLD_MIN, max=THRESHOLD_MAX, step=20)

def update_compounds(*args):
    comp_widget.options = [CLASS_SYMB] + get_comps[comp_class_widget.value]


comp_class_widget.observe(update_compounds, "value")


def update_actis(*args):
    acti_widget.options = [CLASS_SYMB] + get_actis[acti_class_widget.value]


acti_class_widget.observe(update_actis, "value")


def filter_display(comp_class, comp, acti_class, acti, comp_threshold, acti_threshold):
    s_c = to_slice(comp_class, comp)
    s_a = to_slice(acti_class, acti)
    df = dataset.loc[s_c, s_a]
    comp_filter = comp_margin.xs(SELECTORS[1], level=2) >= comp_threshold
    acti_filter = acti_margin.xs(SELECTORS[1], level=2) >= acti_threshold
    display(df.loc[comp_filter,acti_filter])


ipywidgets.interact(
    filter_display,
    comp_class=comp_class_widget,
    comp=comp_widget,
    acti_class=acti_class_widget,
    acti=acti_widget,
    comp_threshold=comp_threshold_widget,
    acti_threshold=acti_threshold_widget,
)


interactive(children=(Dropdown(description='C-Class', options=('*', 'alkaloid', 'phenolic compound', 'polyketi…

<function filter_display at 0x7fcdd6682170>